In [1]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.ts_stock import TsStock
from src.chart import Chart
from src.strategy import FirstStrategy
from src.sina_stock import SinaStock


tsStock = TsStock()
sinaStock = SinaStock()
chart = Chart();


/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [2]:
sma_periods = [8,17,25,43,99,145,318,453]
keyword = '苏泊尔'
init_amount = 100000

stock_info = tsStock.get_stock_info(keyword)
stragety = FirstStrategy(tsStock, keyword , start_date="2010-01-01",sma_periods = sma_periods,sina_realtime=sinaStock)
df = stragety.simulate(start_date="20170101",
                           init_amount=init_amount,
                       max_add_count=4,
                       stop_price_factor=4
                       
                       )

In [3]:
line = chart.get_line_ba(df,init_amount,stock_info)
line.render_notebook()


In [4]:

bs = {}
bs['buy'] = df.loc[df['action']=='buy',['trade_date','unit_price']]
bs['sell'] = df.loc[df['action'] == 'sell', ['trade_date','unit_price']]
df_stop_price = df.loc[df['unit_account']>0,['trade_date','stop_price']]
trade_dates = np.array(df.trade_date)
kdatas = np.array(df.loc[:,['open','close','low','high']])
vols = np.array(df['vol'])
df['bol_up'], df['bol_mi'], df['bol_lo'] = talib.BBANDS(df.close, 20, 2, 2)
sma_dic = {}
for period in sma_periods:
    name = "SMA" + str(period)
#     sma = talib.SMA(np.array(df.close),period)
#     df[name] =sma;
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] =np.array(df[name]).round(2).tolist();


grid_chart= chart.get_grid(
    stock_info,
    trade_dates.tolist(),
    kdatas.tolist(),
    vols.tolist(),
    ma_dic=sma_dic,
    bs = bs, 
    df_stop_price=df_stop_price,
    bol=df[['trade_date','bol_up','bol_mi','bol_lo']] 
)



turnover_chart = chart.get_turnover_chart(df)

adosc_chart = chart.get_adosc_chart(df)

grid_chart.add(
    turnover_chart,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="68%", height="7%"
    ),
)
grid_chart.add(
    adosc_chart,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="75%", height="7%"
    ),
)

grid_chart.render_notebook()


In [5]:
df.tail(10)[['close','high','add_price','stop_price','unit','unit_price','R','P/R','ATR','signal','pre_buy_count']]

,close,high,add_price,stop_price,unit,unit_price,R,P/R,ATR,signal,pre_buy_count
trade_date,,,,,,,,,,,
2019-10-14,75.1044,75.1044,72.243896,66.948119,0.0,0.0,8.464527,0.641933,2.039070,pre_buy,1.0
2019-10-15,76.4100,77.8850,72.273675,69.490490,0.0,0.0,8.464527,0.796177,2.098627,pre_buy,2.0
2019-10-16,77.6358,79.0311,72.284124,70.552995,0.0,0.0,8.464527,0.940993,2.119526,pre_buy,3.0
2019-10-17,77.3269,78.9015,72.287578,70.525367,0.0,0.0,8.464527,0.904500,2.126433,pre_buy,4.0
2019-10-18,76.7389,78.1740,72.276343,70.615244,0.0,0.0,8.464527,0.835033,2.103964,pre_buy,5.0
2019-10-21,77.3069,78.2039,72.265956,70.698342,0.0,0.0,8.464527,0.902137,2.083189,,0.0
2019-10-22,77.2970,77.9348,72.266350,70.695189,0.0,0.0,8.464527,0.900967,2.083978,,0.0
2019-10-23,77.1176,78.6324,72.260151,70.744785,0.0,0.0,8.464527,0.879773,2.071579,,0.0
2019-10-24,75.9715,78.0643,72.271937,70.650494,0.0,0.0,8.464527,0.744373,2.095152,,0.0


In [6]:
df.to_excel('strategy_detail.xlsx')

In [7]:
df.tail(10).to_excel('strategy_detail.xlsx')

In [8]:
75.36 - 0.5 * 2.046


74.337